# Monte Carlo Integration

Ref: Chapter 6, Rizzo

## Basic Monte Carlo Integration

Sampling and integartion are important techniques of statistical inference problems. But, often, the target distributions are too complicated and/or the integrands are too complex or high dimensional for these problems to be solved using basic methods.

Let $p(x)$ be a pdf defined on a sample space $X$. Let $h$ be a function mapping $X$ to $\mathbf{R}$.

The goal is to estimate $E[ h(X) ]$. Normally this is an integral computed by hand ($\mu_p(h) = E[ h(x) ] = \int h(x) p(x) dx$). However, we assume that this case is too complicated to solve that way. It depends on the structure of $h$ and $p$ which can be very difficult to calculate.

In general, we will only assume that $\mu_p (|h|) < \infty$. In other words, we assume a finite first moment. 

The **monte carlo method (MCM)** is based on 2 of the most important results in probability theory:

1. Central limit theorem, CLT ($X_n \xrightarrow{} N(\mu, \sigma)$ as $n \xrightarrow{} \infty$)

2. Law of large numbers, LLN ($\bar{x}_n \xrightarrow{} E [ x ]$ as $n \xrightarrow{} \infty$)

The monte carlo method appeals to the LLN and estimates $ E[ h(X) ]$ by the sample mean of $h(x)$: 

$$\bar{h(x)} = \frac{1}{N} \sum_{i=1}^N h(X_i)$$

and $X_1, X_2, ...$ are iid with pdf $p$. This is simply the average.

Recall LLN ensures that $\bar{h(x)} \xrightarrow{} E [ h(x) ]$ as $N \xrightarrow{} \infty$

We can generate RV from $p$ using inverse method (see **Inverse methods** page). This will generate a set of values $h(x_i)$. With these, we can estimate

$\bar{h(x)} = \frac{h(x_1) + h(x_2) + ... + h(x_n)}{N}$

by the LLN. If the estimate is bad, we need a larger $N$.

The confidence inteval can be defined for $E[ h(x)]$ by appealing to the CLT. The sample variance of $\bar{h(x)}$ is

$$Var(\bar{h(x)}) = \frac{\hat{Var}(h(x))}{N} = \frac{1}{N^2} \sum_{i=1}^N (h(x_i) - \bar{h(x)})^2$$

Because (for instance) if $X \sim (\mu, \sigma^2)$, $E[ \bar{x} ] = \mu$ and $Var(\bar{x}) = \frac{\sigma^2}{n}$.

The CLT tells us that the approximate distribution of $\bar{h(x)}$ is approximately normal ($ N(E(h(x)), \frac{var(h(x))}{N})$)

Therefore, the ($1 - \alpha$) CI for $E[ h(x)]$ is approximately 

$$\bar{h(x)} \pm z_{\alpha/2} \sqrt{\frac{\hat{Var}(h(x))}{N}}$$


**Example:** Estimate $E[ h(x)] = \int_{-\infty}^\infty \frac{1}{1 + \exp (-x)} \frac{1}{\sqrt{ (2\pi) (1.43)}} \exp (\frac{-(x - 1.5)^2}{2.86}) dx$ and give a 95% CI for $E[h(x)]$.

Using $\int h(x) p(x) dx$

Note that $h(x) = \frac{1}{1 + e^{-x}}$ and is the inverse of the logit function.

$$p(x) = \frac{1}{\sqrt{(2\pi)(1.43)}} \exp (\frac{-(x - 1.5)^2}{2.36})$$

is the pdf of the $N(1.5, 1.43)$. To estimate this integral, 

Step 1: Generate $Z_1, Z_2, ..., Z_{n} \sim N(1.5, 1.43)$ where $n$ is just a large number, say 10000.

Step 2: Calculate $\hat{I} = \frac{1}{n} \sum_{i=1}^n \frac{1}{1 + e^{-Z_i}}$ as $n \xrightarrow{} \infty$, $\hat{I} \xrightarrow{} E[h(x)]$

In [3]:
import numpy as np

n = 1000

# We can also estimate this using base numpy. See `inversion methods` page.
g = np.random.normal(1.5,1.43,n)

integral = (1 / n) * np.sum(1 / (1 + np.exp(g)))
integral

0.2511457485713463

For the 95\% CI for $E[h(X)]$, we will need an estimate of the variance. The corresponding variance is 

$$Var(\hat{I}) = \frac{1}{n} \sum_{i=1}^n (\frac{1}{1 + e^{-Z_i}} - \hat{I})^2$$

So, the 95\% CI for $E[h(x)]$ is $\hat{I} \mp 1.96 \sqrt{\frac{Var(\hat{I})}{n}}$


## Calculating arbitrary integrals

This is a special case also known as "sample mean". or "crude" method.

To calculate more general integrals, we generate a random variable from [a,b] and find summation.

$$\int_a^b h(x) dx = \int_a^b \frac{h(x)p(x)}{p(x)} dx = \int_a^b f(x) p(x) dx$$

where $f(x) = \frac{h(x)}{p(x)}$. This re-writes the integral as an expectation against some density $f$, generates from that density, and looks at the sample mean as before.

For definite integrals over a finite interval, the uniform distribution will suffice (but you may not get a good accuracy).

To calculate $I = \int_a^b h(x)dx$, we can proceed as

$$I = \int_a^b h(x) dx = (b-a)\int_a^b h(x) \frac{1}{b-a}dx = (b-a)\int_a^b h(x) f_{unif}(a,b)dx $$



**Example:** Estimate $\theta = \int_0^2 e^{-x^2} dx$

$\theta$ can be rewritten as

$$\theta = (2 - 0) \int_0^2 e^{-x^2} \frac{1}{2 - 0} dx = 2 \int_0^2 e^{-x^2} \frac{1}{2} dx$$

Step 1: Sample $U_1, ... U_n \sim Unif(0,2)$

Step 2: Estimate $\theta$ by $\hat{\theta} = \frac{2}{n} \sum_{i=1}^n e^{-U_i^2}$

In [5]:
import numpy as np
n = 1000
u = np.random.uniform(0,2,n)

theta = (2/n) * np.sum(np.exp(-1 * u**2))
theta

0.8659644587536548

 **Example** Estimate $\theta = \int_0^{2\pi} \sin(x \cos(x)) dx$
 
  $$\theta = (2\pi - 0) \int_0^{2\pi} \sin(x \cos(x)) \frac{1}{2\pi - 0} dx = 2\pi \int_0^{2\pi} \sin(x \cos(x)) \frac{1}{2 \pi} dx$$

  Step 1: Generate $unif(0, 2 \pi)$

  Step 2: Build $\frac{2\pi}{n} \sum_{i=1}^n sin(U_i cos(U_i))$

In [19]:
import numpy as np
n = 1000000
u = np.random.uniform(0,2*np.pi,n)
theta = (2*np.pi/n) * np.sum(np.sin(u * np.cos(u)))
theta

-1.0438478105216198

**Example:** What about $\int_{-\infty}^\infty \sin(x \cos(x))dx$?

Here, the uniform approx won't work because we cannot depend on an evaluation of $b - a$.

The way to solve this is discussed below.

## Infinite integrals

Use any distribution defined on the real line and do a similar approximation.
So, rewrite the integral as 

$$\int_{-\infty}^{\infty} h(x) dx = \int_{-\infty}^{\infty} \frac{h(x) p(x)}{p(x)} dx$$

where $p(x)$ is the pdf of a distribution defined over $\mathbf{R}$, and $h$ in any function.

Then, generate $X_i$ from $p(x)$ and estimate the integral $\int_{-\infty}^\infty h(x) dx$ by using 

$$\frac{1}{n} \sum_{i=1}^n \frac{h(x_i)}{p(x_i)} $$

per the problem earlier.

**Example** Back to the problem from earlier: compute $\int_{-\infty}^\infty \sin(x \cos(x))dx$?

The standard normal is supported over $\mathbf{R}$. So, use the standard normal pdf for the estimation.

Step 1: Generate $X_1, ..., X_n \sim N(0,1)$

Step 2: Calcualte the average, $\frac{1}{n} \sum_{i=1}^n \frac{\sin (x_i \cos(x_i))}{\phi(x_i)}$

where $\phi(.)$ is the pdf of the standard normal.

In [20]:
import numpy as np

n=1000
g = np.random.normal(0,1,n)

def gaussian_pdf(x):
    sigm = 1
    mu = 0
    return (1 / np.sqrt(2 * np.pi * sigm**2)) * np.exp(-1 * (x - mu)**2 / (2 * sigm**2))

np.sum( np.sin(g * np.cos(g)) / gaussian_pdf(g))

-20.41176776122647

## Higher-dimensional integrals

Monte carlo methods become particularly attractive when we consider integration in higher dimensions

$$I = \int_0^1 \int_0^1 \int_0^1 f(x, y, z) dx dy dz$$

$I$ can be estimated by

$\hat{I} = \frac{1}{n} \sum_{i=1}^n f(x_i, y_i, z_i)$ where $(x_i, y_i, z_i)$ is a random sequence of points in the unit cube $[0, 1] \times [0, 1] \times [0, 1]$.
For example,  $X_i \sim Unif(0,1), Y_i \sim Unif(0,1), Z_i \sim Unif(0,1)$.

In total, we need $3 n$ random numbers in order to generate the $N$ random points.

**Example:** $I = \int_0^1 \int_0^1 x y \exp (- x^2 y) dx dy$

Here, $f(x,y) = x y \exp (- x^2 y)$. So, 

$$\hat{I} = \frac{1}{n} \sum_{i=1}^n x_i y_i \exp (- x_i^2 y_i)$$





In [24]:
import numpy as np

n = 100000
u = np.random.uniform(0,1,n)
u2 = np.random.uniform(0,1,n)

approx = (1 / n) * np.sum(u * u2 * np.exp(- u * u2**2))
real = 1 / (2 * np.e)

approx, real

(0.18458428310422237, 0.18393972058572117)

Recall that the exact value of $I$ is $I = \frac{1}{2e}$ We see a close approximation.